<div style="text-align:center"><span style="font-size:2em; font-weight: bold;">Lecture 5—Optimization</span></div>

# Data science: Logistic regression

## Derivation

**Linear formulation**

$$\mathcal L=\prod_{i=1}^n p_i^{y_i}(1-p_i)^{1-y_i}$$
$$\mathcal L=\prod_{i=1}^n F(x_i'\beta)^{y_i}(1-F(x_i'\beta))^{1-y_i}$$
$$F(x)=\frac{1}{1+e^{-x}}$$
$$\ln\mathcal L=\sum_{i=1}^n y_i \ln{F(x_i'\beta)}+(1-y_i)\ln{(1-F(x_i'\beta))}$$
$$\ln\mathcal L=\left[\ln{F(\beta'\mathbf X')}\right]y+\left[\ln{(\mathbf{1}'-F(\beta'\mathbf X'))}\right](1-y)$$

In [1]:
expit = lambda x: 1/(1+np.exp(-x))
def loglike(x,y,b):
    Fx = expit(b.T@x.T)
    return np.log(Fx)@y+np.log(1-Fx)@(1-y)

**Gradient**
$$\frac{d\ln\mathcal L}{d\beta}=\mathbf X'\text{diag}\left(\frac{f(\mathbf X\beta)}{F(\mathbf X\beta)}\right)y-\mathbf X'\text{diag}\left(\frac{f(\mathbf X\beta)}{\mathbf 1-F(\mathbf X\beta)}\right)(1-y)$$
$$\frac{d\ln\mathcal L}{d\beta}=\mathbf X'\text{diag}\left(\frac{f(\mathbf X\beta)(1-F(\mathbf X\beta))}{(1-F(\mathbf X\beta))F(\mathbf X\beta)}\right)y-\mathbf X'\text{diag}\left(\frac{f(\mathbf X\beta)F(\mathbf X\beta)}{F(\mathbf X\beta)(1-F(\mathbf X\beta))}\right)(1-y)$$
$$\frac{d\ln\mathcal L}{d\beta}=\mathbf X'\left[\text{diag}\left(1-F(\mathbf X\beta)\right)y-\text{diag}\left(F(\mathbf X\beta)\right)(1-y)\right]$$
$$\frac{d\ln\mathcal L}{d\beta}=\mathbf X'\left[\text{diag}\left(y-F(\mathbf X\beta)y-F(\mathbf X\beta)+F(\mathbf X\beta)y)\right)\right]\mathbf 1$$
$$\frac{d\ln\mathcal L}{d\beta}=\mathbf X'\left[\text{diag}\left(y-F(\mathbf X\beta)\right)\right]\mathbf 1$$
$$\frac{d\ln\mathcal L}{d\beta}=\mathbf X'\left[y-F(\mathbf X\beta)\right]$$

In [2]:
def gradient(x,y,b):
    Fx = expit(x@b)
    return x.T@(y-Fx)

**Hessian**
$$\frac{d}{d\beta}\frac{d\ln\mathcal L}{d\beta}'=\frac{d}{d\beta}\left[y'-F(\beta'\mathbf X')\right]\mathbf X$$
$$\frac{d^2\ln\mathcal L}{d\beta d\beta'}=-\mathbf X'\left[\text{diag}\left(f(\mathbf X\beta)\right)\right]\mathbf X$$

In [3]:
def hessian(x,y,b):
    Fx = expit(x@b)
    fx = Fx*(1-Fx)
    return -x.T@np.diagflat(fx.flatten())@x

**Theorem** Crammer-Rao Lower Bound

Assume
$\mathcal{L}$ is continuous and differentiable. For any unbiased estimator $\hat\theta$, the variance is bounded below by
$$\text{Var}\left[\hat\theta\right]\ge\left[-\text{E}\left[\frac{d^2\ln{\mathcal{L}}}{d\theta d\theta'}\right]\right]^{-1}$$


# Programming--Numerical Optimization Strategies

## Grid search

Search over a given parameter space. Check every possible option for the optimum value

In [4]:
import numpy as np
from itertools import product

def grid_search(func,space,maximize=False):
    vstates = [(x,func(x)) for x in space]
    vstates.sort(key=lambda x: x[1])
    return vstates[-1][0] if maximize else vstates[0][0]

x = np.linspace(0,10,1000).tolist()
func = lambda x: (x[0]-4.0001)**2*(x[1]-6.0001)**2
grid_search(func,product(x,x))

(4.004004004004004, 5.995995995995996)

## Gradient descent

Walk along the slope of the curve by steps proportional to the opposite of the size of the gradient. 

In [5]:
def gradient_descent(func,gradient,init_x:np.ndarray,learning_rate:float=0.005,max_reps:int=10000,maximize=False):
    x = init_x.copy()
    for i in range(max_reps):
        gx = gradient(x)
        x0 = x.copy()
        flast = func(x)
        x += gx*learning_rate if maximize else -gx*learning_rate
        if (func(x)<flast and maximize and i>2) or (func(x)>flast and (not maximize) and i>2): 
            x = x0
            break
    return x


## Newton's method

Use a zero finding algorithm on the gradient to isolate where the gradient is flat, i.e., where the maximum or minimum values of the function are located.

In [6]:
def newton(gradient,hessian,init_x:np.ndarray,max_reps:int=100,tolerance:float=1e-16):
    x = init_x.copy()
    for i in range(max_reps):
        update = -np.linalg.solve(hessian(x),gradient(x))
        x += update
        if np.abs(update).sum()<tolerance:
            return (x,i)
    raise Exception('Newton did not converge')

## Complete code

In [7]:
from cleands import *

class logistic_regressor(linear_model):
    def __fit__(self,x,y):
        params,self.iters = self.__max_likelihood__(np.zeros(self.n_feat))
        return params
    @property
    def vcov_params(self):return self.__vcov_params_lnL__()
    def evaluate_lnL(self,pred):return self.y.T@np.log(pred)+(1-self.y).T@np.log(1-pred)
    def _gradient_(self,coefs):return self.x.T@(self.y-expit(self.x@coefs))
    def _hessian_(self,coefs):
        Fx = expit(self.x@coefs)
        return -self.x.T@np.diagflat((Fx*(1-Fx)).values)@self.x
    def predict(self,target):return expit(target@self.params)

class LogisticRegressor(logistic_regressor,broom_model):
    def __init__(self,x_vars:list,y_var:str,data:pd.DataFrame,*args,**kwargs):
        super(LogisticRegressor,self).__init__(data[x_vars],data[y_var],*args,**kwargs)
        self.x_vars = x_vars
        self.y_var = y_var
        self.data = data
    def _glance_dict_(self):
        return {'mcfadden.r.squared':self.r_squared,
                'adjusted.r.squared':self.adjusted_r_squared,
                'self.df':self.n_feat,
                'resid.df':self.degrees_of_freedom,
                'aic':self.aic,
                'bic':self.bic,
                'log.likelihood':self.lnL,
                'deviance':self.deviance,
                'resid.var':self.ssq}

In [8]:
from cleands import *

In [9]:
## Data generation
df = pd.DataFrame(np.random.normal(size=(10000,4)),columns=['x1','x2','x3','y'])
df['y'] += df[['x1','x2','x3']]@np.random.uniform(size=(3,))
df['y'] = (df['y']>0).astype(int)

In [10]:
## Run the model
model = LogisticRegressor(*add_intercept(['x1','x2','x3'],'y',df))

In [11]:
## See table
model.tidy

,variable,estimate,std.error,t.statistic,p.value
0,(intercept),-0.040441047002977966,0.023764327618393647,-1.701754312277554,0.08883253096659166
1,x1,1.4332323065954125,0.03213013724995693,44.607101907021374,0.0
2,x2,0.36865797971370656,0.024428122910568378,15.091539413952002,6.682399566127589e-51
3,x3,0.2126205510431411,0.02430480096440112,8.748088550676192,2.5199437419579284e-18


In [12]:
model.glance

,mcfadden.r.squared,adjusted.r.squared,self.df,resid.df,aic,bic,log.likelihood,deviance,resid.var
,0.290335,0.290122,4,9996,10573.608328,10602.449689,-5282.804164,3291.184253,0.177378


In [13]:
model.iters

6

# Programming challenges

## Recursive partitioning trees

Write a class that implements a recursive partitioning algorithm. Use our common machine learning code.

In [2]:
import numpy as np
import pandas as pd

from cleands import *

In [19]:
class rpart(prediction_model):
    def __init__(self,x,y,max_level,level=''):
        super(rpart,self).__init__(x,y)
        self.max_level = max_level
        self.level = level
        if len(level)+1==max_level:
            self.RSS = np.sum((y-y.mean())**2)
            self.split_var = None
            self.split_value = None
            self.left = None
            self.right = None
            return
        xvars = np.arange(self.n_feat)
        outp = []
        for i in xvars:
            outp += [self.__calc_RSS_and_split__(x[:,i])]
        outp = np.array(outp)
        var = outp[:,0].argmin()
        self.RSS = outp[var,0]
        self.split_var = var
        self.split_value = outp[var,1]
        self.left = rpart(x[x[:,var]<=self.split_value,:],
                          y[x[:,var]<=self.split_value],
                          max_level=max_level,level=level+'L')
        self.right = rpart(x[x[:,var]>self.split_value,:],
                           y[x[:,var]>self.split_value],
                           max_level=max_level,level=level+'R')
    def __calc_RSS_and_split__(self,var):
        vmin = var.min()
        vmax = var.max()
        width = (vmax-vmin)/50
        outp = []
        for split in np.linspace(vmin+width,vmax-width,48):
            left = self.y[var<=split]
            right = self.y[var>split]
            rssleft = ((left-left.mean())**2).sum() if left.shape[0]>0 else 0
            rssright = ((right-right.mean())**2).sum() if right.shape[0]>0 else 0
            outp += [(rssleft+rssright,split)]
        outp = np.array(outp)
        return outp[outp[:,0].argmin(),:]
    def __str__(self):
        if self.left==None and self.right==None:
            outp = '{0} RSS: {1}; Prediction: {2}\n'.format(self.level,self.RSS,self.y.mean())
        else:
            outp = '{0} Variable: {1}; Split: {2}\n'.format(self.level,self.split_var,self.split_value)
            outp += str(self.left)
            outp += str(self.right)
        return outp
    def predict(self,newx):
        if self.left==None and self.right==None:
            return np.full(shape=(self.n_obs,),fill_value=self.y.mean())
        outp = np.zeros((self.n_obs,))
        outp[newx[:,self.split_var]<=self.split_value] = self.left.predict(newx[newx[:,self.split_var]<=self.split_value,:])
        outp[newx[:,self.split_var]>self.split_value] = self.right.predict(newx[newx[:,self.split_var]>self.split_value,:])
        return outp

In [20]:
n = 100000
x = np.random.normal(size=(n,3))
y = np.random.normal(size=(n,))
y += x@np.random.uniform(size=(3,))

In [21]:
model = rpart(x,y,max_level=4)
print(model)

 Variable: 1; Split: 0.06722073230816239
L Variable: 0; Split: 0.04190451804433071
LL Variable: 1; Split: -0.9272965214734512
LLL RSS: 11258.0579094251; Prediction: -1.2464887099489177
LLR RSS: 21352.893167776114; Prediction: -0.6249683143625077
LR Variable: 1; Split: -0.9525177334572534
LRL RSS: 10289.8242154867; Prediction: -0.48541370693014513
LRR RSS: 20563.77181411077; Prediction: 0.18994876396663316
R Variable: 0; Split: 0.013017941549896328
RL Variable: 1; Split: 1.0408036522075315
RLL RSS: 19890.905838076607; Prediction: -0.09475091949267307
RLR RSS: 9422.842241014518; Prediction: 0.527967196456722
RR Variable: 1; Split: 1.0878281728396062
RRL RSS: 19756.629222256193; Prediction: 0.7276542771137003
RRR RSS: 8631.180825251475; Prediction: 1.3468529849134876



In [22]:
model = rpart(x,y,max_level=3)
print(model)

 Variable: 1; Split: 0.06722073230816239
L Variable: 0; Split: 0.04190451804433071
LL RSS: 34942.42777568047; Prediction: -0.8348517859766391
LR RSS: 33385.08358248156; Prediction: -0.027069479137251044
R Variable: 0; Split: 0.013017941549896328
RL RSS: 31308.796069180178; Prediction: 0.09873705059500838
RR RSS: 30272.08545823423; Prediction: 0.911504373431792



In [24]:
(model.residuals**2).sum()

129908.39288557644

In [25]:
34942.42777568047+33385.08358248156+31308.796069180178+30272.08545823423

129908.39288557645

## Quaternions

The Quaternions are a generalization of complex numbers. Where the complex numbers have two components, $a$ and $b$, for a number $a+bi$, the Quaternions have four parts $a, b, c$ and $d$: $$a+bi+cj+dk$$

The Quaternions have four basic operations: addition, subtraction, multiplication, and the inverse. Also write a str representation function. Your job is to write a quaternion class which implements these operations. You can learn how to perform these operations on the Quaternions' wikipedia page.

In [49]:
class quaternion:
    def __init__(self,a,b,c,d):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
    def __str__(self):
        return '{0}+{1}i+{2}j+{3}k'.format(self.a,self.b,self.c,self.d)
    def __add__(self,other):
        if type(other)==quaternion:
            return quaternion(self.a+other.a,self.b+other.b,self.c+other.c,self.d+other.d)
        else:
            return quaternion(self.a+other,self.b,self.c,self.d)
    def __radd__(self,other):
        return self+other
    def __mul__(self,other):
        if type(other)==quaternion:
            return quaternion(self.a*other.a-self.b*other.b-self.c*other.c-self.d*other.d,
                              self.a*other.b+self.b*other.a+self.c*other.d-self.d*other.c,
                              self.a*other.c-self.b*other.d+self.c*other.a+self.d*other.b,
                              self.a*other.d+self.b*other.c-self.c*other.b+self.d*other.a)
        else:
            return quaternion(self.a*other,self.b*other,self.c*other,self.d*other)
    def __rmul__(self,other):
        return self*other
    def __sub__(self,other):
        return self+-1*other
    def __rsub__(self,other):
        return other+-1*self
    def __invert__(self):
        norm2 = self.a**2+self.b**2+self.c**2+self.d**2
        return quaternion(self.a/norm2,-self.b/norm2,-self.c/norm2,-self.d/norm2)

In [37]:
print(quaternion(5,0,3,0))

5+0i+3j+0k


In [38]:
print(quaternion(5,0,3,0)+quaternion(2,-1,-2,4))

7+-1i+1j+4k


In [39]:
print(quaternion(5,0,3,0)+4)

9+0i+3j+0k


In [40]:
print(4+quaternion(5,0,3,0))

9+0i+3j+0k


In [42]:
print(quaternion(5,0,3,0)*quaternion(2,-1,-2,4))

16+7i+-4j+23k


In [43]:
print(quaternion(5,0,3,0)*4)

20+0i+12j+0k


In [44]:
print(4*quaternion(5,0,3,0))

20+0i+12j+0k


In [46]:
print(quaternion(5,0,3,0)-quaternion(2,-1,-2,4))

3+1i+5j+-4k


In [47]:
print(quaternion(5,0,3,0)-4)

1+0i+3j+0k


In [48]:
print(4-quaternion(5,0,3,0))

-1+0i+-3j+0k


In [50]:
print(~quaternion(5,0,3,0))

0.14705882352941177+0.0i+-0.08823529411764706j+0.0k


In [51]:
print(~quaternion(5,0,3,0)*quaternion(5,0,3,0))

1.0+0.0i+-5.551115123125783e-17j+0.0k


In [52]:
print(quaternion(5,0,3,0)*~quaternion(5,0,3,0))

1.0+0.0i+-5.551115123125783e-17j+0.0k
